# Word2Vec CBOW and Skipgram
## written by: [Jehwan Kim](github.com/kreimben)
## date: 10th Feb 2024 ~ 12th Feb 2024
## corpus data: [kaggle](https://www.kaggle.com/datasets/junbumlee/kcbert-pretraining-corpus-korean-news-comments?resource=download)

In [ ]:
import logging  # Setting up the loggings to monitor gensim

from gensim.models import Word2Vec
from soynlp import DoublespaceLineCorpus
from soynlp.tokenizer import MaxScoreTokenizer

from src.util import *

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt='%H:%M:%S', level=logging.INFO)

# Data Preprocessing

Of course there are great library `KoNLPy`, But I found more accurate library [`soynlp`](https://github.com/lovit/soynlp) so I will use that.
soynlp learns new words automatically and calculate words statistically.
soynlp use `Cohesion score`, `Branching Entropy` and `Accessor Variety` internally.

In [ ]:
corpus = DoublespaceLineCorpus("data.txt", iter_sent=True, num_sent=1_000_000)
len(corpus)

In [ ]:
%%time
# Awesome way to extract the words. https://github.com/lovit/soynlp

from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor_v2
import pickle

word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(corpus)
words = word_extractor.extract()
with open('words.pkl', 'wb') as handle:
    pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%time

cohesion_score = {word: score.cohesion_forward for word, score in words.items()}

noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(corpus)  # list of str like

noun_scores = {noun: score.score for noun, score in nouns.items()}
combined_scores = {noun: score + cohesion_score.get(noun, 0)
                   for noun, score in noun_scores.items()}
combined_scores.update(
    {subword: cohesion for subword, cohesion in cohesion_score.items()
     if not (subword in combined_scores)}
)
# Initializing MaxScoreTokenizer with trained scores
tokenizer = MaxScoreTokenizer(scores=combined_scores)

In [ ]:
tokenizer.tokenize('혼전임신은 미리 조심하지 못한 여자 잘못이 크다')

In [ ]:
import pickle

with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from soynlp.vectorizer import BaseVectorizer

vectorizer = BaseVectorizer(
    tokenizer=tokenizer,
    min_tf=0,
    max_tf=10000,
    min_df=0,
    max_df=1.0,
    stopwords=None,
    lowercase=True,
    verbose=True
)

corpus.iter_sent = False
x = vectorizer.fit_transform(corpus)

In [ ]:
with open('vectorizer.pkl', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# to check the score of each method.
word_score = word_extractor.extract()

## Cohension
Cohesion은 문자열을 글자단위로 분리하여 부분문자열(substring)을 만들 때 왼쪽부터 문맥을 증가시키면서 각 문맥이 주어졌을 때 그 다음 글자가 나올 확률을 계산하여 누적곱을 한 값이다.

![](img1.png)

예를 들어 “연합뉴스가”라는 문자열이 있는 경우, 각 부분문자열의 cohesion은 다음과 같다. 한 글자는 cohesion을 계산하지 않는다.

![](img2.png)

하나의 단어를 중간에서 나눈 경우, 다음 글자를 예측하기 쉬우므로 조건부확률의 값은 크다. 하지만 단어가 종료된 다음에 여러가지 조사나 결합어가 오는 경우에는 다양한 경우가 가능하므로 조건부확률의 값이 작아진다. 따라서 cohesion값이 가장 큰 위치가 하나의 단어를 이루고 있을 가능성이 높다.

In [ ]:
word_score["문"].cohesion_forward

In [ ]:
word_score["문재"].cohesion_forward

In [ ]:
word_score["문재인"].cohesion_forward

In [ ]:
word_score["문재인은"].cohesion_forward

In [ ]:
word_score["문재인이"].cohesion_forward

## Branching Entropy
Branching Entropy는 조건부 확률의 값이 아니라 확률분포의 엔트로피값을 사용한다. 
만약 하나의 단어를 중간에서 끊으면 다음에 나올 글자는 쉽게 예측이 가능하다. 
즉, 여러가지 글자 중 특정한 하나의 글자가 확률이 높다. 따라서 엔트로피값이 0에 가까운 값으로 작아진다. 
하지만 하나의 단어가 완결되는 위치에는 다양한 조사나 결합어가 올 수 있으므로 여러가지 글자의 확률이 비슷하게 나오고 따라서 엔트로피값이 높아진다.

In [ ]:
word_score["문"].right_branching_entropy

In [ ]:
# '핵실' 다음에는 항상 '험'만 나온다.
word_score["문재"].right_branching_entropy

In [ ]:
word_score["문재인"].right_branching_entropy

In [ ]:
word_score["문재인은"].right_branching_entropy

# Accessor Variety
Accessor Variety는 확률분포를 구하지 않고 단순히 특정 문자열 다음에 나올 수 있는 글자의 종류만 계산한다. 
글자의 종류가 많다면 엔트로피가 높아지리 것이라고 추정하는 것이다.

In [ ]:
word_score["문"].right_accessor_variety

In [ ]:
# '핵실' 다음에는 항상 '험'만 나온다.
word_score["문재"].right_accessor_variety

In [ ]:
word_score["문재인"].right_accessor_variety

In [ ]:
word_score["문재인은"].right_accessor_variety

We can assume the corpus is from social network and better `Max Score` tokenising for this corpus cuz it's not cultured writing.

# Max Score Tokenising

최대 점수 토큰화(max score tokenizing)는 띄어쓰기가 되어 있지 않는 긴 문자열에서 가능한 모든 종류의 부분문자열을 만들어서 가장 점수가 높은 것을 하나의 토큰으로 정한다. 
이 토큰을 제외하면 이 위치를 기준으로 전체 문자열이 다시 더 작은 문자열들로 나누어지는데 이 문자열들에 대해 다시 한번 가장 점수가 높은 부분문자열을 찾는 것을 반복한다. 

In [ ]:
%%time

from src.stopwords import *

# Dataset example
count = 500_000
data = get_data()
tokenised_sentences = []

while count:
    if count % 10_000 == 0: print(f'{count}', end=' ')
    temp = ltokenizer.tokenize(next(data))
    approve = []
    for word in temp:
        if not word in STOP_WORDS: approve.append(word)
    tokenised_sentences.append(approve)
    count -= 1

In [ ]:
len(tokenised_sentences)

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [ ]:
%%time
cbow_model = Word2Vec(sentences=tokenised_sentences,
                      sg=0,
                      hs=1,
                      min_count=20,
                      window=10,
                      vector_size=100,
                      sample=6e-5,
                      alpha=0.03,  # initial learning rate
                      min_alpha=0.0007,  #minimum learning rate
                      negative=20,
                      workers=cores - 1)

In [ ]:
%%time
skipgram_model = Word2Vec(sentences=tokenised_sentences,
                          sg=1,
                          hs=1,
                          min_count=20,
                          window=10,
                          vector_size=100,
                          sample=6e-5,
                          alpha=0.03,
                          min_alpha=0.0007,
                          negative=20,
                          workers=cores - 1)

In [ ]:
cbow_model.init_sims(replace=True)
skipgram_model.init_sims(replace=True)

In [ ]:
skipgram_model.wv.most_similar(positive='문재인')

In [ ]:
cbow_model.wv.most_similar(positive='문재인')

In [ ]:
cbow_model.wv.doesnt_match(['김정은', '북한', '문재인', '트럼프'])

In [ ]:
skipgram_model.wv.doesnt_match(['김정은', '북한', '문재인', '트럼프'])

In [ ]:
cbow_model.wv.similarity('트럼프', '김정은')

In [ ]:
skipgram_model.wv.similarity('김정은', '북한')

In [ ]:
cbow_model.wv.most_similar(positive=['김정은', '남한'], negative=['문재인'])

In [ ]:
skipgram_model.wv.most_similar(positive=['김정은', '남한'], negative=['문재인'])

In [ ]:
cbow_model.predict_output_word(['새끼'], topn=5)

In [ ]:
skipgram_model.predict_output_word(['새끼'], topn=10)

In [ ]:
# save the model
cbow_model.wv.save_word2vec_format('./model/cbow.model')
skipgram_model.wv.save_word2vec_format('./model/skipgram.model')

## Visualisation about model

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from gensim.models import KeyedVectors

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

plt.rc('font', family='D2Coding')  # Set proper font


def show_tsne(vocab_show, X_show):
    tsne = TSNE(n_components=2)
    X = tsne.fit_transform(X_show)

    df = pd.DataFrame(X, index=vocab_show, columns=['x', 'y'])
    fig = plt.figure()
    fig.set_size_inches(30, 20)
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(df['x'], df['y'])

    for word, pos in df.iterrows():
        ax.annotate(word, pos, fontsize=10)

    plt.xlabel("t-SNE 특성 0")
    plt.ylabel("t-SNE 특성 1")
    plt.show()


def show_pca(vocab_show, X_show):
    # PCA 모델을 생성합니다
    pca = PCA(n_components=2)
    pca.fit(X_show)
    # 처음 두 개의 주성분으로 숫자 데이터를 변환합니다
    x_pca = pca.transform(X_show)

    plt.figure(figsize=(30, 20))
    plt.xlim(x_pca[:, 0].min(), x_pca[:, 0].max())
    plt.ylim(x_pca[:, 1].min(), x_pca[:, 1].max())
    for i in range(len(X_show)):
        plt.text(x_pca[i, 0], x_pca[i, 1], str(vocab_show[i]),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xlabel("첫 번째 주성분")
    plt.ylabel("두 번째 주성분")
    plt.show()
    return None

def visualise(model_name):
    model = KeyedVectors.load_word2vec_format(model_name)
    
    vocab = model.index_to_key
    X = model[vocab]
    
    # sz개의 단어에 대해서만 시각화
    sz = 800
    X_show = X[:sz, :]
    vocab_show = vocab[:sz]
    
    show_tsne(vocab_show, X_show)
    show_pca(vocab_show, X_show)
    return None

In [ ]:
visualise('./model/cbow.model')

In [ ]:
visualise('./model/skipgram.model')